In [11]:
import torch as t
import torch.nn as nn
import argparse

from imports import *
from encoder import Encoder as encoder
from decoder import Decoder as decoder

# Taking a simple sent that will be passed into the transformer. 
sent: t.Tensor = t.randint(0, 8, (4,))
out_sent: t.Tensor = t.randint(0, 8, (4,))

print(f"Input sent: {sent}")
print(f"Output sent: {out_sent}")

class Transformer(nn.Module):
    def __init__(self, num_heads: int, sent: t.Tensor, out_sent: t.Tensor):
        super(Transformer, self).__init__()
        self.encoder = encoder(num_heads, sent)
        self.decoder = decoder(num_heads, out_sent)

    def output(self, decoder_output: t.Tensor) -> t.Tensor:
        linear_output = nn.Linear(512, 8)(decoder_output)  # 8 is the vocabulary size
        return linear_output

    def forward(self) -> t.Tensor:
        encoder_output = self.encoder.forward()
        decoder_output = self.decoder.forward(encoder_output) 
        return self.output(decoder_output)
# parser = argparse.ArgumentParser(
ce_loss = nn.CrossEntropyLoss()
num_heads = 2

model = Transformer(num_heads, sent, out_sent)
model.train()
predicted_logits = model.forward()
ce_loss_none = nn.CrossEntropyLoss(reduction='none')
loss_contributions = ce_loss_none(predicted_logits.view(4, 8), out_sent)
print(f"Contribution of each word to the loss: {loss_contributions}")

Input sent: tensor([0, 1, 4, 1])
Output sent: tensor([5, 6, 5, 4])
torch.Size([1, 4, 1024])
Contribution of each word to the loss: tensor([2.0407, 2.7079, 2.0915, 2.7559], grad_fn=<NllLossBackward0>)


/Users/maheepchaudhary/pytorch/Projects/Transformer-from-Scratch/encoder.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return t.tensor(final_sent, dtype = t.float32)
/Users/maheepchaudhary/pytorch/Projects/Transformer-from-Scratch/decoder.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return t.tensor(final_sent, dtype = t.float32)


In [12]:
loss_contributions.mean().backward(create_graph=True)

tensor([[ 9.5959e-05,  2.1622e-04,  8.3552e-05,  ...,  1.4928e-04,
          4.8936e-05,  1.4928e-04],
        [ 4.0750e-04,  2.9007e-04,  3.7078e-04,  ...,  1.5501e-04,
          2.0285e-04,  1.5501e-04],
        [-2.8666e-06,  8.7148e-05, -3.7111e-06,  ...,  7.6415e-05,
          4.4379e-07,  7.6415e-05],
        ...,
        [ 6.4185e-05, -1.3429e-05,  7.6700e-05,  ...,  1.1356e-04,
          5.1338e-05,  1.1356e-04],
        [ 1.7569e-04,  4.4332e-04,  1.9392e-04,  ...,  6.6561e-04,
          1.1842e-04,  6.6561e-04],
        [ 1.7234e-05, -9.1292e-05,  2.2618e-05,  ..., -4.3527e-05,
          1.8944e-05, -4.3527e-05]], grad_fn=<CopyBackwards>)

In [8]:
model

Transformer(
  (encoder): Encoder(
    (W_q): Linear(in_features=512, out_features=1024, bias=True)
    (W_k): Linear(in_features=512, out_features=1024, bias=True)
    (W_v): Linear(in_features=512, out_features=1024, bias=True)
    (W_o): Linear(in_features=1024, out_features=512, bias=True)
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (fc1): Linear(in_features=512, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=512, bias=True)
    (relu): ReLU()
  )
  (decoder): Decoder(
    (W_q): Linear(in_features=512, out_features=1024, bias=True)
    (W_k): Linear(in_features=512, out_features=1024, bias=True)
    (W_v): Linear(in_features=512, out_features=1024, bias=True)
    (W_q_m): Linear(in_features=512, out_features=1024, bias=True)
    (W_k_m): Linear(in_features=512, out_features=1024, bias=True)
    (W_v_m): Linear(in_features=512, out_features=1024, bias=True)
    (W_o): Linear(in_features=1024, out_features=512, bias=Tru